## Installing External Libraries

In [2]:
!pip install efficientnet_pytorch --quiet

## Libraries

In [3]:
################ Importing Libraries ################

import os
import torch
import random
import torchvision
import numpy as np
import pandas as pd
import torch.nn as nn
from tqdm import tqdm
import albumentations as A
import torch.cuda.amp as amp
from torchinfo import summary
import matplotlib.pyplot as plt
from PIL import Image, ImageFile
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score
from timeit import default_timer as timer
from efficientnet_pytorch import EfficientNet
from torch.utils.data import Dataset, DataLoader
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
!mkdir Models

pd.set_option('expand_frame_repr', False)
print(torch.__version__, torchvision.__version__)

1.13.0 0.14.0


## Configurations

In [4]:
## Configuration Settings ##

class Config:
        
    EPOCHS = 20
    IMG_SIZE = 512
    RESOLUTION = 456
    ES_PATIENCE = 2
    WEIGHT_DECAY = 0.001
    VAL_BATCH_SIZE = 32 * 2
    RANDOM_STATE = 1994
    LEARNING_RATE = 5e-5
    TRAIN_BATCH_SIZE = 32
    MEAN = (0.485, 0.456, 0.406)
    STD = (0.229, 0.224, 0.225)
    TRAIN_COLS = ["image_name", "patient_id", "sex", "age_approx", "anatom_site_general_challenge",
                 "target", "tfrecord"]
    TEST_COLS = ["image_name", "patient_id", "sex", "age_approx", "anatom_site_general_challenge"]
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

    ################ Setting paths to data input ################
    
    data_2020 = "/kaggle/input/jpeg-melanoma-512x512/"
    train_folder_2020 = data_2020 + "train/"
    test_folder_2020 = data_2020 + "test/"
    test_csv_path_2020 = data_2020 + "test.csv"
    train_csv_path_2020 = data_2020 + "train.csv"
    submission_csv_path = data_2020 + "sample_submission.csv"

## Utilities

In [5]:
## Helper Utilities

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience.
       Directly borrowed from https://github.com/Bjarten/early-stopping-pytorch/blob/master/pytorchtools.py"""
    def __init__(self, path, patience=7, verbose=False, delta=0, trace_func=print):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print            
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0
            
    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(obj=model.state_dict(), f=self.path)
        self.val_loss_min = val_loss
        
def plot_loss_curves(results):
    """
    Function to plot training & validation loss curves & validation AUC
    """
    loss = results['train_loss']
    valid_loss = results['valid_loss']
    # Get the accuracy values of the results dictionary (training and test)
    valid_auc = results['valid_auc']
    # Figure out how many epochs there were
    epochs = range(len(results['train_loss']))
    # Setup a plot 
    plt.figure(figsize=(15, 7))
    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, loss, label='train_loss')
    plt.plot(epochs, valid_loss, label='valid_loss')
    plt.title('Loss'); plt.xlabel('Epochs');plt.legend()
    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, valid_auc, label='valid_auc')
    plt.title('AUC Score'); plt.xlabel('Epochs'); plt.legend();


## Dataset

In [6]:
## Creating Dataset classes to load the images

ImageFile.LOAD_TRUNCATED_IMAGES = True

class DatasetRetriever(nn.Module):
    def __init__(self, df, tabular_features=None, use_tabular_features=False,
                augmentations=None, is_test=False):
        self.df = df
        self.tabular_features = tabular_features
        self.use_tabular_features = use_tabular_features
        self.augmentations = augmentations
        self.is_test = is_test
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        image_path = self.df['image_path'].iloc[index]
        image = Image.open(image_path)
        image = np.array(image)
        if self.augmentations is not None:
            augmented = self.augmentations(image=image)
            image = augmented['image']
        image = np.transpose(image, (2,0,1)).astype(np.float32)
        image = torch.tensor(image, dtype=torch.float)
        if self.use_tabular_features:
            if len(self.tabular_features) > 0 and self.is_test == False:
                tabular_features = np.array(self.df.iloc[index][self.tabular_features].values, dtype=np.float32)
                targets = self.df.target[index]
                return {"image": image, "tabular_features": tabular_features, "targets": torch.tensor(targets, dtype=torch.long)}
            elif len(self.tabular_features) > 0 and self.is_test == True:
                tabular_features = np.array(self.df.iloc[index][self.tabular_features].values, dtype=np.float32)
                return {"image": image, "tabular_features": tabular_features}
        else:
            if self.is_test == False:
                targets = self.df.target[index]
                return {"image" : image, "targets": torch.tensor(targets, dtype=torch.long)}
            elif self.is_test == True:
                return {"image": image}

## Training & Evaluate Loops

In [7]:
def train_one_epoch(model, dataloader, loss_fn, optimizer, device, scaler, use_tabular_features=False):
    train_loss = 0
    model.train()
    for batch, data in enumerate(dataloader):
        if use_tabular_features:
            if batch == 0:
                print("Using meta features")
            data["image"], data["meta_features"], data['targets'] = data["image"].to(device, dtype=torch.float), \
                    data["meta_features"].to(device, dtype=torch.float), data['targets'].to(device, dtype=torch.float)
        else:
            if batch == 0:
                print("Not using meta features")
            data["image"], data['targets'] = data["image"].to(device, dtype=torch.float), data['targets'].to(device, dtype=torch.float)
        optimizer.zero_grad()
        with amp.autocast():
            y_logits = model(data['image']).squeeze(dim=0)
            loss = loss_fn(y_logits, data["targets"].view(-1,1))
        train_loss += loss.item()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()        
    train_loss = train_loss / len(dataloader)
    return train_loss

def validate_one_epoch(model, dataloader, loss_fn, device, use_tabular_features=False):
    valid_loss, final_predictions = 0, []
    model.eval()
    with torch.inference_mode():
        for batch, data in enumerate(dataloader):
            if use_tabular_features:
                if batch == 0:
                    print("Using meta features")
                data["image"], data["meta_features"], data['targets'] = data["image"].to(device, dtype=torch.float), \
                    data["meta_features"].to(device, dtype=torch.float), data['targets'].to(device, dtype=torch.float)
            else:
                if batch == 0:
                    print("Not using meta features")
                data["image"], data['targets'] = data["image"].to(device, dtype=torch.float), data['targets'].to(device, dtype=torch.float)    
            y_logits = model(data['image']).squeeze(dim=0)
            loss = loss_fn(y_logits, data["targets"].view(-1,1))
            valid_loss += loss.item()
            valid_probs = torch.sigmoid(y_logits).detach().cpu().numpy().tolist()
            final_predictions.extend(valid_probs)
    valid_loss = valid_loss / len(dataloader)
    return valid_loss, final_predictions

def train(model, train_dataloader, valid_dataloader, loss_fn, optimizer, scheduler,
          device, scaler, epochs, es_patience, model_save_path, validation_targets):
    results = {"train_loss": [], "valid_loss": [], "valid_auc": []}
    
    early_stopping = EarlyStopping(patience=es_patience, verbose=True, path=model_save_path)
    
    for epoch in tqdm(range(epochs)):
        train_loss = train_one_epoch(model=model, dataloader=train_dataloader,
                                    loss_fn=loss_fn, optimizer=optimizer,
                                    device=device, scaler=scaler)
        
        valid_loss, valid_predictions = validate_one_epoch(model=model, 
                                    dataloader=valid_dataloader, loss_fn=loss_fn, device=device)
        
        valid_predictions = np.vstack(valid_predictions).ravel()
        
        valid_auc = roc_auc_score(y_score=valid_predictions, 
                                  y_true=validation_targets)
        scheduler.step(valid_auc)
        
        early_stopping(valid_loss, model)
        
        if early_stopping.early_stop:
            print(f"Early Stopping")
            break
            
        model.load_state_dict(torch.load(model_save_path))
        print(f"Epoch : {epoch+1} | "
              f"train_loss : {train_loss:.4f} | "
              f"valid_loss : {valid_loss:.4f} | "
              f"valid_auc : {valid_auc:.4f} ")
        results['train_loss'].append(train_loss)
        results['valid_loss'].append(valid_loss)
        results['valid_auc'].append(valid_auc)
    return results

## Augmentations

In [8]:
## Setting Training & Validation Augmentations

training_augmentations = A.Compose([A.CoarseDropout(p=0.6),
                                    A.RandomRotate90(p=0.6),
                                    A.Flip(p=0.4),
                                    A.OneOf([A.RandomBrightnessContrast(brightness_limit=0.2,
                                                                       contrast_limit=0.3),
                                            A.HueSaturationValue(hue_shift_limit=20,
                                                                sat_shift_limit=60,
                                                                val_shift_limit=50)], p=0.7),
                                    A.OneOf([A.GaussianBlur(),
                                            A.GaussNoise()], p=0.65),
                                    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.35, rotate_limit=45, p=0.5),
                                    A.OneOf([A.OpticalDistortion(p=0.3),
                                            A.GridDistortion(p=0.1),
                                            A.PiecewiseAffine(p=0.3)], p=0.7),
    A.Normalize(mean=Config.MEAN, std=Config.STD,
                max_pixel_value=255.0, always_apply=True)
])

validation_augmentations = A.Compose([A.Normalize(mean=Config.MEAN, std=Config.STD, 
                                        max_pixel_value=255.0,always_apply=True)
                                     ])

## Model

In [9]:
class Model(nn.Module):
    def __init__(self,model_name='efficientnet-b5',pool_type=F.adaptive_avg_pool2d):
        super().__init__()
        self.pool_type = pool_type
        self.model_name = model_name
        self.backbone = EfficientNet.from_pretrained(model_name)
        in_features = getattr(self.backbone,'_fc').in_features
        self.classifier = nn.Linear(in_features,1)

    def forward(self,x):
        features = self.pool_type(self.backbone.extract_features(x),1)
        features = features.view(x.size(0),-1)
        return self.classifier(features)

## Reading the Data

In [10]:
## Looking at the Data

train_df = pd.read_csv(Config.train_csv_path_2020,
                       usecols=Config.TRAIN_COLS)
print(f"Number of Data points & Columns in Training dataset are - {train_df.shape}\n")

test_df = pd.read_csv(Config.test_csv_path_2020,
                       usecols=Config.TEST_COLS)
print(f"Number of Data points & Columns in 2020 Testing dataset are - {test_df.shape}\n")

print(f"Train Dataset Columns: {Config.TRAIN_COLS}\n")
print(f"Test Dataset Columns: {Config.TEST_COLS}\n")

print(f"Distribution of Target feature in training dataset are: \n{train_df['target'].value_counts(normalize=True) * 100}\n")
print("*"*70 + "\n")

print(f"Distribution of sex feature in training dataset are:\n{train_df['sex'].value_counts(normalize=True, dropna=False) * 100}\n")
print("*"*50 + "\n")
print(f"Distribution of sex feature in testing dataset are:\n{test_df['sex'].value_counts(normalize=True, dropna=False) * 100}\n")

print(f"Distribution of anatom_site_general_challenge feature in training dataset are:\n\n{train_df['anatom_site_general_challenge'].value_counts(normalize=True, dropna=False) * 100}\n")
print("*"*70 + "\n")
print(f"Distribution of anatom_site_general_challenge feature in testing dataset are:\n\n{test_df['anatom_site_general_challenge'].value_counts(normalize=True, dropna=False) * 100}")
print("*"*70 + "\n")

print(f"------------------- Missing values distribution in training dataset-------------------:\n{train_df.isnull().sum()}\n")
print(f"-------------------Missing values distribution in testing dataset-------------------:\n{test_df.isnull().sum()}\n")

print('\n\nViewing Training Dataset below\n')
print(train_df.head())
print("*"*100 + "\n")
print('\n\nViewing Testing Dataset below\n')
print(test_df.head())

Number of Data points & Columns in Training dataset are - (33126, 7)

Number of Data points & Columns in 2020 Testing dataset are - (10982, 5)

Train Dataset Columns: ['image_name', 'patient_id', 'sex', 'age_approx', 'anatom_site_general_challenge', 'target', 'tfrecord']

Test Dataset Columns: ['image_name', 'patient_id', 'sex', 'age_approx', 'anatom_site_general_challenge']

Distribution of Target feature in training dataset are: 
0    98.237034
1     1.762966
Name: target, dtype: float64

**********************************************************************

Distribution of sex feature in training dataset are:
male      51.560708
female    48.243072
NaN        0.196220
Name: sex, dtype: float64

**************************************************

Distribution of sex feature in testing dataset are:
male      56.956838
female    43.043162
Name: sex, dtype: float64

Distribution of anatom_site_general_challenge feature in training dataset are:

torso              50.851295
lower extrem

In [11]:
def create_folds(train_df=train_df):
    train_df = train_df.loc[train_df['tfrecord'] != -1].reset_index(drop=True)
    train_df['fold'] = train_df['tfrecord'] % 5
    return train_df

In [12]:
## Creating Image_Path for each images in 2019 & 2020 training datasets
train_df['image_path'] = os.path.join(Config.train_folder_2020) + train_df['image_name'] + ".jpg"
test_df['image_path'] = os.path.join(Config.test_folder_2020) + test_df['image_name'] + ".jpg"

train_df.head()

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,target,tfrecord,image_path
0,ISIC_2637011,IP_7279968,male,45.0,head/neck,0,0,/kaggle/input/jpeg-melanoma-512x512/train/ISIC...
1,ISIC_0015719,IP_3075186,female,45.0,upper extremity,0,0,/kaggle/input/jpeg-melanoma-512x512/train/ISIC...
2,ISIC_0052212,IP_2842074,female,50.0,lower extremity,0,6,/kaggle/input/jpeg-melanoma-512x512/train/ISIC...
3,ISIC_0068279,IP_6890425,female,45.0,head/neck,0,0,/kaggle/input/jpeg-melanoma-512x512/train/ISIC...
4,ISIC_0074268,IP_8723313,female,55.0,upper extremity,0,11,/kaggle/input/jpeg-melanoma-512x512/train/ISIC...


In [13]:
def run_model(fold, train_df):
    train_df = create_folds(train_df=train_df)
    train_data = train_df.loc[train_df['fold'] != fold].reset_index(drop=True)
    valid_data = train_df.loc[train_df['fold'] == fold].reset_index(drop=True)
    validation_targets = valid_data['target']
    train_dataset = DatasetRetriever(df=train_data, tabular_features=None, use_tabular_features=False, 
                     augmentations=training_augmentations, is_test=False)
    valid_dataset = DatasetRetriever(df=valid_data, tabular_features=None, use_tabular_features=False, 
                     augmentations=validation_augmentations, is_test=False)
    training_dataloader = DataLoader(dataset=train_dataset, batch_size=Config.TRAIN_BATCH_SIZE, 
                                     shuffle=True, num_workers=os.cpu_count())
    validation_dataloader = DataLoader(dataset=valid_dataset, batch_size=Config.VAL_BATCH_SIZE,
                                       shuffle=False, num_workers=os.cpu_count())
    seed_everything(Config.RANDOM_STATE)
    if torch.cuda.device_count() in (0,1):
        model = Model().to(Config.DEVICE)
    elif torch.cuda.device_count() > 1:
        model = Model().to(Config.DEVICE)
        model = nn.DataParallel(model)
    loss = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.AdamW(params=model.parameters(), lr=Config.LEARNING_RATE, weight_decay=Config.WEIGHT_DECAY)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, 
                                    mode='max', factor=0.2, patience=2, 
                                    threshold=1e-3,verbose=True)
    scaler = amp.GradScaler()
    start_time = timer()
    model_save_path = f"Models/efficientnet_b5_checkpoint_fold_{fold}.pt"
    model_results = train(model=model, train_dataloader=training_dataloader, 
                            valid_dataloader=validation_dataloader,
                            loss_fn = loss, optimizer=optimizer, scheduler=scheduler,
                            device=Config.DEVICE, scaler=scaler,
                            epochs=Config.EPOCHS, es_patience=2, 
    model_save_path=model_save_path, validation_targets=validation_targets)
    end_time = timer()
    print(f"Total training time: {end_time-start_time:.3f} seconds")

In [14]:
def validate(fold, train_df):
    full_train_df = create_folds(train_df)
    valid_data = full_train_df.loc[full_train_df['fold'] == fold].reset_index(drop=True)
    validation_targets = valid_data['target']
    valid_dataset = DatasetRetriever(df=valid_data, tabular_features=None, use_tabular_features=False, 
                     augmentations=validation_augmentations, is_test=False)
    validation_dataloader = DataLoader(dataset=valid_dataset, batch_size=Config.VAL_BATCH_SIZE,
                                       shuffle=False, num_workers=os.cpu_count())
    valid_predictions = []
    if torch.cuda.device_count() in (0,1):
        EfficientNetB0_trained_model = Model().to(Config.DEVICE)
    elif torch.cuda.device_count() > 1:
        EfficientNetB0_trained_model = Model().to(Config.DEVICE)
        EfficientNetB0_trained_model = nn.DataParallel(EfficientNetB0_trained_model)
    EfficientNetB0_trained_model.load_state_dict(torch.load(f"/kaggle/working/Models/efficientnet_b5_checkpoint_fold_{fold}.pt"))
    EfficientNetB0_trained_model.eval()
    with torch.inference_mode():
        for batch, data in enumerate(validation_dataloader):
            data["image"], data['targets'] = data["image"].to(Config.DEVICE, dtype=torch.float), data['targets'].to(Config.DEVICE, dtype=torch.float)    
            y_logits = EfficientNetB0_trained_model(data['image']).squeeze(dim=0)
            valid_probs = torch.sigmoid(y_logits).detach().cpu().numpy().tolist()
            valid_predictions.extend(valid_probs)
    valid_auc = roc_auc_score(y_true=validation_targets, y_score=valid_predictions)
    print(valid_auc)

In [15]:
def predict_on_test(fold):
    test_dataset = DatasetRetriever(df=test_df, tabular_features=None, use_tabular_features=False, 
                     augmentations=validation_augmentations, is_test=True)
    test_dataloader = DataLoader(dataset=test_dataset, batch_size=Config.VAL_BATCH_SIZE,
                                       shuffle=False, num_workers=os.cpu_count())
    test_predictions = []
    if torch.cuda.device_count() in (0,1):
        EfficientNetB0_trained_model = Model().to(Config.DEVICE)
    elif torch.cuda.device_count() > 1:
        EfficientNetB0_trained_model = Model().to(Config.DEVICE)
        EfficientNetB0_trained_model = nn.DataParallel(EfficientNetB0_trained_model)
    EfficientNetB0_trained_model.load_state_dict(torch.load(f"/kaggle/working/Models/efficientnet_b5_checkpoint_fold_{fold}.pt"))   
    EfficientNetB0_trained_model.eval()
    with torch.inference_mode():
        for batch, data in enumerate(test_dataloader):
            data["image"] = data["image"].to(Config.DEVICE, dtype=torch.float)    
            y_logits = EfficientNetB0_trained_model(data['image']).squeeze(dim=0)
            test_probs = torch.sigmoid(y_logits).detach().cpu().numpy()
            test_predictions.extend(test_probs)
    submission_df = pd.read_csv(Config.submission_csv_path)
    test_predictions = [test_predictions[img].item() for img in range(len(test_predictions))]
    submission_df['target'] = test_predictions
    submission_df.to_csv("submission.csv", index=False)

In [16]:
run_model(fold=0, train_df=train_df)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b5-b6417697.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b5-b6417697.pth


  0%|          | 0.00/117M [00:00<?, ?B/s]

Loaded pretrained weights for efficientnet-b5


  0%|          | 0/20 [00:00<?, ?it/s]

Not using meta features
Not using meta features
Validation loss decreased (inf --> 0.071890).  Saving model ...


...

Epoch : 4 | train_loss : 0.0706 | valid_loss : 0.0656 | valid_auc : 0.8971 
Not using meta features
Not using meta features
EarlyStopping counter: 1 out of 2


 25%|██▌       | 5/20 [4:28:53<13:30:42, 3242.85s/it]

Epoch : 5 | train_loss : 0.0683 | valid_loss : 0.0673 | valid_auc : 0.9015 
Not using meta features
Not using meta features


 25%|██▌       | 5/20 [5:22:17<16:06:52, 3867.49s/it]

EarlyStopping counter: 2 out of 2
Early Stopping
Total training time: 19337.441 seconds


In [17]:
validate(fold=0, train_df=train_df)

Loaded pretrained weights for efficientnet-b5
0.897142585912284


In [18]:
predict_on_test(fold=0)

Loaded pretrained weights for efficientnet-b5


This model gave a score of 0.8928 on private leadeboard & 0.8942 on public leaderboard.